# Notebook for datasets generation

In [1]:
import os         
import pathlib
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm
from os import listdir
from pathlib import Path
from astropy.io import fits
from os.path import isfile, join

import matplotlib.pyplot as plt
%matplotlib widget

datapath = Path("/data01/home/baroncelli/phd/DATA")
os.environ["DATA"] = str(datapath)
from RTAscience.cfg.Config import Config

from RTAscience.lib.RTAUtils import get_pointing
from rtapipe.lib.datasource.photometry import Photometry
from rtapipe.lib.plotting.photometryplot import PhotometrySinglePlot, PhotometrySubPlots


Bad key "text.kerning_factor" on line 4 in
/data01/home/baroncelli/.conda/envs/bphd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


## Directories

In [2]:
currentdir = pathlib.Path().absolute()
currentdir

PosixPath('/data01/home/baroncelli/phd/repos/phd/rtapipe/analysis')

In [3]:
obsdir = datapath.joinpath("obs")
obsdir

PosixPath('/data01/home/baroncelli/phd/DATA/obs')

In [4]:
outdir = currentdir.joinpath("notebook_dataset_generation_for_models_output")
outdir

PosixPath('/data01/home/baroncelli/phd/repos/phd/rtapipe/analysis/notebook_dataset_generation_for_models_output')

## Utility function

In [5]:
def getInput(dataFolder, outdir, howMany=-1):

    cfg = Config(dataFolder.joinpath("config.yaml"))
    
    # get pointing from template
    runid = cfg.get('runid')
    template =  os.path.join(datapath, f'templates/{runid}.fits')
    pointing = get_pointing(template)
    
    # get files
    datafiles = [join(dataFolder, f) for f in listdir(dataFolder) 
                    if isfile(join(dataFolder, f)) and ".yaml" not in f and ".log" not in f and "IRF" not in f] 
    
    # get simulation parameters
    sim_params = []
    for datafile in datafiles:
        
        conf = {
            'input_file': str(datafile),
            'output_dir': str(outdir),
            'simtype' : cfg.get('simtype'),
            'runid' : cfg.get('runid'),
            't_window_start': 0,
            't_window_stop': cfg.get('tobs'),
            'e_window_start': cfg.get('emin'),
            'e_window_stop': cfg.get('emax'),
            'onset' : cfg.get('onset')
        }
        sim_params.append(conf)

        if len(sim_params) >= howMany:
            break
            
            
    print(f"Found: {len(sim_params)} files, pointing is: {pointing}")

    return sim_params, pointing


## Background only 

In [6]:
inputd = obsdir.joinpath("autoencoder/run0406_ID000126_OK_bkg_only")

In [7]:
outd = outdir.joinpath("run0406_ID000126_OK_bkg_only")

In [8]:
photometry = Photometry()

photometry_params = {
    
    # integration parameters for time
    't_window_size': 10,
    't_window_step': 10,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : None,
    'region_radius': 1.0,
    
    # Other settings
    'plot' : 1,
    'override': 1
}

In [12]:
sim_params, pointing = getInput(inputd, outdir=outd, howMany=101)
print("pointing: ",pointing)
photometry_params['pointing'] = pointing

Found: 101 files, pointing is: (33.057, -51.841)
pointing:  (33.057, -51.841)


In [13]:
print("photometry_params")
_ = [print(f"\t{k}: {v}") for k,v in photometry_params.items()]
print("sim_params[0]")
_ = [print(f"\t{k}: {v}") for k,v in sim_params[0].items()]

photometry_params
	t_window_size: 10
	t_window_step: 10
	e_window_size: 0.001
	e_window_step: 0.001
	pointing: (33.057, -51.841)
	region_radius: 1.0
	plot: 1
	override: 1
sim_params[0]
	input_file: /data01/home/baroncelli/phd/DATA/obs/autoencoder/run0406_ID000126_OK_bkg_only/bkg000018.fits
	output_dir: /data01/home/baroncelli/phd/repos/phd/rtapipe/analysis/notebook_dataset_generation_for_models_output/run0406_ID000126_OK_bkg_only
	simtype: bkg
	runid: run0406_ID000126
	t_window_start: 0
	t_window_stop: 1800
	e_window_start: 0.03
	e_window_stop: 0.15
	onset: 0


In [14]:
# singleplot = PhotometrySinglePlot(title="Multiple trials")
for i in tqdm(range(1, 101)):
    csv_data_t = photometry.integrate(photometry_params, sim_params[i], integration="t").pop(0)
    # _ = singleplot.addData(csv_data_t, photometry_params, sim_params[i], [f"trial {i}"], "t")


100%|██████████| 100/100 [01:49<00:00,  1.09s/it]


## GRB after onset

In [15]:
inputd = obsdir.joinpath("autoencoder/run0406_ID000126_OK_bkg_src")

In [16]:
outd = outdir.joinpath("run0406_ID000126_OK_bkg_src")

In [17]:
photometry = Photometry()

photometry_params = {
    
    # integration parameters for time
    't_window_size': 10,
    't_window_step': 10,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : None,
    'region_radius': 1.0,
    
    # Other settings
    'plot' : 1,
    'override': 1
}

In [18]:
sim_params, pointing = getInput(inputd, outdir=outd, howMany=101)
print("pointing: ",pointing)
photometry_params['pointing'] = pointing

Found: 101 files, pointing is: (33.057, -51.841)
pointing:  (33.057, -51.841)


In [19]:
print("photometry_params")
_ = [print(f"\t{k}: {v}") for k,v in photometry_params.items()]
print("sim_params[0]")
_ = [print(f"\t{k}: {v}") for k,v in sim_params[0].items()]

photometry_params
	t_window_size: 10
	t_window_step: 10
	e_window_size: 0.001
	e_window_step: 0.001
	pointing: (33.057, -51.841)
	region_radius: 1.0
	plot: 1
	override: 1
sim_params[0]
	input_file: /data01/home/baroncelli/phd/DATA/obs/autoencoder/run0406_ID000126_OK_bkg_src/ebl000114.fits
	output_dir: /data01/home/baroncelli/phd/repos/phd/rtapipe/analysis/notebook_dataset_generation_for_models_output/run0406_ID000126_OK_bkg_src
	simtype: grb
	runid: run0406_ID000126
	t_window_start: 0
	t_window_stop: 1800
	e_window_start: 0.03
	e_window_stop: 0.15
	onset: 100


In [20]:
# singleplot = PhotometrySinglePlot(title="Multiple trials")
for i in tqdm(range(1,101)):
    csv_data_t = photometry.integrate(photometry_params, sim_params[i], integration="t").pop(0)  
    # _ = singleplot.addData(csv_data_t, photometry_params, sim_params[i], [f"trial {i}"], "t")

100%|██████████| 100/100 [01:49<00:00,  1.10s/it]
